## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabweichung für die aufgenommenen Messwerte aus der Datei "merkmaleRoh.csv" der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit einer Länge von 80ms (siehe hierzu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung). Bei dem Beschleunigungssensor sind dies die Messwerte der beschleunigung von den 3-Achsen X, Y sowie Z (Abb. 1). Des Weiteren werden die Gier-Nick-Roll Winkel in rad in einem Zeitfenster erfasst. Für die Berechnung des Mittelwerts, Varianz und Standardabweichung über die Messwerte wird die Form der abgespeicherten Messwerte geändert, weil diese als Python Objekte verwaltet werden mit welchen nicht gerechnet werden kann. Zu den Messwertreihen wird jeweils eine CSV Datei erstellt. Eine Reihe stellt dann in dieser Datei eine Messreihe zu einem Zeitfenster dar. Bei der ausführung dieses Notebooks werden deshalb zusätzlich folgende CSV Dateien erstellt:

X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2  
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad  
Nick Messwerte in rad  
Roll Messwerte in rad   
Zeitstempel der Messwerte in ns  

<img src="../daten/abbildungen/sensorOrientierung.png" alt="Android Smartphone Sensor Orientierung" width="200px" />
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: siehe Bildverzeichnis Abb. 2 in der README.md Datei

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [3]:
dataset = '41'

In [4]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond,Lokations Zeitstempel in Unixzeit,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Zeitstempel Messwertdaten anfordern in Unixzeit,Start der Messwerterfassung in ns seit Start der JVM,Erster Zeitstempel der Teilliste in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Speicherzeitpunkt der Merkmale in Unixzeit
count,5.940000e+03,5940.000000,5940.000000,5940.000000,5940.000000,5.940000e+03,5940.000000,5.940000e+03,5.940000e+03,5.940000e+03,5.940000e+03,5.940000e+03,5.940000e+03,5.940000e+03,5.940000e+03
mean,1.528299e+12,53.620429,10.132441,14.086600,0.630095,1.528299e+12,32.686195,3.978221e+12,1.528299e+12,3.923616e+12,3.977625e+12,3.978350e+12,1.506374e+07,3.978496e+12,1.528299e+12
std,6.336331e+05,0.004857,0.006974,3.271286,0.281096,6.336315e+05,10.310478,6.336412e+11,6.336266e+05,6.252329e+11,6.336443e+11,6.336338e+11,1.200439e+07,6.336265e+11,6.336254e+05
min,1.528298e+12,53.611855,10.122842,5.004000,0.114018,1.528298e+12,18.000000,2.856502e+12,1.528298e+12,2.856286e+12,2.856286e+12,2.856718e+12,5.699570e+05,2.856982e+12,1.528298e+12
25%,1.528299e+12,53.616320,10.125096,12.060000,0.436005,1.528299e+12,26.000000,3.418525e+12,1.528299e+12,3.384804e+12,3.418008e+12,3.418707e+12,2.426916e+06,3.418915e+12,1.528299e+12
50%,1.528299e+12,53.620193,10.133554,14.400000,0.588218,1.528299e+12,29.000000,3.994610e+12,1.528299e+12,3.933726e+12,3.994109e+12,3.994717e+12,1.000184e+07,3.994825e+12,1.528299e+12
75%,1.528300e+12,53.624798,10.137742,16.380000,0.773692,1.528300e+12,36.000000,4.531558e+12,1.528300e+12,4.479266e+12,4.531002e+12,4.531711e+12,2.999632e+07,4.531889e+12,1.528300e+12
max,1.528300e+12,53.628616,10.145172,23.688000,2.455056,1.528300e+12,89.000000,5.063213e+12,1.528300e+12,5.047202e+12,5.062674e+12,5.063534e+12,3.287369e+07,5.063849e+12,1.528300e+12


In [5]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940 entries, 0 to 5939
Data columns (total 19 columns):
Zeitstempel in Unixzeit                                               5940 non-null int64
Breitengrad                                                           5940 non-null float64
Laengengrad                                                           5940 non-null float64
Geschwindigkeit in km/h                                               5940 non-null float64
Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond          5940 non-null float64
Lokations Zeitstempel in Unixzeit                                     5940 non-null int64
Z-Achse Beschleunigungswerte in m/s^2                                 5940 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 5940 non-null object
Nick Messwerte in rad                                                 5940 non-null object
Zeitstempel der Messwerte in ns                                       59

In [ ]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

In [ ]:
# Die Anzahl der Messwerte sollte immer gleich sein, ansonsten werden fehlende Spalten mit NaN Werten gefüllt bei der
# späteren Abspeicherung der erfassten Messwerte wie z.B. in die Datei x-Achse-Beschleunigungswerte.csv.
# Um die Berechnung von den Merkmalen Varianz und Mittelwert unteranderem zu beschleunigen ist es wichtig sogenannte Außenseiter
# aus den Datensätzen zu entfernen.
# Dafür muss auch das Bild zu einer Reihe aus der merkmaleRoh.csv Datei gelöscht werden.
# Dies wird erreicht indem alle Pfade der Bilder geladen werden.
# Dann können Reihen sowie Bilder gelöscht werden mit mehr Messwerten als ein bestimmter Grenzwert.
print(featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23])

################### Entferne Außenseiter mit Anzahl der Messwerte > Grenzwert ###################
#### Warnung löschen der zugeschnittennen Bilder kann nicht rückgängig gemacht werden!
#threshold  = 23 
#imagePaths = []
# Hier können die Datensätze ausgewählt werden
#datasets = ['15','16','17','18','19','20','21','22']
# Die Pfade zu den Ordnern in welchem sich die Bilder befinden
#paths = []
#for dataset in datasets: # Für jeden Datensatz merke Pfad
#    paths.append("C:/Users/morro/Documents/datenRoh/" + dataset + "/zugeschnitten/")
#for path in paths: # Für jeden Pfad hole die Namen der Ordner
#    folders = os.listdir(path)
#    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
#        filesPath = path + folder + "/"
#        files = os.listdir(filesPath)
#        for name in files: # Ein Dateiname aus diesem Ordner
#            if "jpg" not in name:
#                continue
#            imagePaths.append(filesPath + name) # Path wird geladen in Liste
#toRemoveImagesIndex = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > threshold].index.tolist()
#for i in toRemoveImagesIndex:
#    print(imagePaths[i])
#    os.remove(imagePaths[i])
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]
featuresDf.drop(unwantedRows.index, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

In [ ]:
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [ ]:
featuresDf.info()

In [ ]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [ ]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

In [ ]:
accXList[12] # zeige Liste

In [ ]:
np.array(accXList[1]).dtype

In [ ]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(abs(np.array(accXList[0]).astype(float))).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())

In [ ]:
accXDf = pandas.concat(accList, ignore_index=True)

In [ ]:
accXDf.head()

In [ ]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.T.mean().head(20)

In [ ]:
accXDf.shape

In [21]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [7]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940 entries, 0 to 5939
Data columns (total 89 columns):
0     5940 non-null float64
1     5940 non-null float64
2     5940 non-null float64
3     5940 non-null float64
4     5940 non-null float64
5     5940 non-null float64
6     5940 non-null float64
7     5940 non-null float64
8     5940 non-null float64
9     5940 non-null float64
10    5940 non-null float64
11    5940 non-null float64
12    5940 non-null float64
13    5940 non-null float64
14    5940 non-null float64
15    5940 non-null float64
16    5940 non-null float64
17    5940 non-null float64
18    5935 non-null float64
19    5926 non-null float64
20    5880 non-null float64
21    5812 non-null float64
22    5672 non-null float64
23    5388 non-null float64
24    5079 non-null float64
25    4681 non-null float64
26    4227 non-null float64
27    3755 non-null float64
28    3319 non-null float64
29    2961 non-null float64
30    2667 non-null float64
31    2407 non-null float

In [8]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
0,1.53828,0.99954,3.17388,3.41181,0.49604,0.53810,0.32496,0.78601,1.22555,0.87730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.85446,-0.20665,-0.07723,0.33926,-0.37236,-0.37574,0.91950,0.81559,0.69254,0.32236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.65433,-0.65223,-0.91605,-0.01444,-0.01600,0.19101,-1.18852,-1.18645,-0.32113,-0.31579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.07753,-0.97502,-0.83478,-0.31230,-0.21154,-0.20761,1.87945,0.04506,-2.14720,0.69261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.48617,-0.82370,-2.24529,-1.63206,-1.01869,-0.29595,-0.36516,-0.17063,-0.39560,-1.39872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [23]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [ ]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [ ]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [24]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [25]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
#featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXDf.T.mean()
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
#featuresDf['Mittelwert Roll in rad'] = rollDf.T.mean()

In [26]:
#featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

In [27]:
featuresDf.columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h',
       'Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond',
       'Lokations Zeitstempel in Unixzeit',
       'Z-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2', 'Nick Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Zeitstempel Messwertdaten anfordern in Unixzeit',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Erster Zeitstempel der Teilliste in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Speicherzeitpunkt der Merkmale in Unixzeit',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
   

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [13]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [14]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [15]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [ ]:
featuresDf['Gier-Aenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [ ]:
accXDf[14957:14961]

In [ ]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].iloc[10]

In [28]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
#featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
#featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [ ]:
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'].head()

In [ ]:
calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])

In [29]:
# Berechne Standardabweichung
#featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
#featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [ ]:
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'].head()

In [ ]:
accXDf.info()

In [ ]:
accXDf.head()

In [30]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_'+ dataset + '/merkmale.csv')

In [31]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmale.csv")

In [32]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
#accXDf.index.names = ["index"]
#accXDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/xAchseBeschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/nickMesswerte.csv')
#rollDf.index.names = ["index"]
#rollDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/rollMesswerte.csv')
#azimuthDf.index.names = ["index"]
#azimuthDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/azimuthMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/sensoreventZeitstempel.csv')

In [ ]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

In [111]:
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'].iloc[27906-30]

2.2395099999999997

In [109]:
featuresDf.iloc[27906-31]

Zeitstempel in Unixzeit                                                                                   1527877755443
Breitengrad                                                                                                     53.6138
Laengengrad                                                                                                     10.1379
Geschwindigkeit in km/h                                                                                           8.964
Z-Achse Beschleunigungswerte in m/s^2                                 3.31214 -1.45266 -0.85647 -6.07319 -1.51277 2....
Y-Achse Beschleunigungswerte in m/s^2                                 -5.42426 7.26983 9.74416 4.87022 -6.38916 -2.3...
Nick Messwerte in rad                                                 0.3893 -0.75311 -0.81811 -0.79712 0.53538 0.18...
Zeitstempel der Messwerte in ns                                       5862970 13858263 21855014 29849734 37840548 45...
Anzahl der Messwerte                    

In [102]:
featuresDf[featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2']]

KeyError: '[ 0.60104  1.50801  0.7393  ...,  0.81991  0.91911  0.71639] not in index'

In [57]:
featuresDf[featuresDf['Anzahl der Messwerte'] == featuresDf['Anzahl der Messwerte'].max()]

,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Z-Achse Beschleunigungswerte in m/s^2,Y-Achse Beschleunigungswerte in m/s^2,Nick Messwerte in rad,Zeitstempel der Messwerte in ns,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Zeitstempel Messwertdaten anfordern in Unixzeit,...,Speicherzeitpunkt der Merkmale in Unixzeit,Mittelwert Y-Achse Beschleunigung in m/s^2,Mittelwert Z-Achse Beschleunigung in m/s^2,Mittelwert Nick in rad,Varianz Y-Achse Beschleunigung in m/s^2,Varianz Z-Achse Beschleunigung in m/s^2,Varianz Nick in rad,Standardabweichung Y-Achse Beschleunigung in m/s^2,Standardabweichung Z-Achse Beschleunigung in m/s^2,Standardabweichung Nick in rad
Zeitstempel in Unixzeit,,,,,,,,,,,,,,,,,,,,,
1527871509062,53.626556,10.027216,5.004,-3.03752 -2.72623 0.6619 0.2198 1.09744 1.9162...,3.73035 4.11822 0.84982 -0.4632 0.56443 0.4897...,-0.50581 -0.51976 -0.05175 0.083 -0.02236 -0.0...,2489033 10482763 18476493 26473818 34470100 42...,89,25534077246068,1527871509328,...,1527871509364,0.061497,-0.167370,-0.032344,8.38072,10.71274,0.10781,2.89495,3.27303,0.32834
1527871509569,53.626556,10.027216,5.004,1.47978 2.69973 1.62354 -0.38691 -4.11345 -5.4...,-0.28844 1.13599 1.98382 1.80012 3.5326 4.6945...,0.04344 -0.07728 -0.16167 -0.18855 -0.59838 -0...,5112656 13097064 21099284 29096243 37097787 45...,89,25534578345068,1527871509829,...,1527871509871,-0.246005,-0.113654,-0.077562,19.09061,39.84095,0.26680,4.36928,6.31197,0.51653
1527873740791,53.665268,10.078330,5.004,-0.35468 0.04737 1.06002 -0.52924 -0.52854 -1....,-1.38725 -0.87037 -1.21106 2.08359 2.0813 0.22...,0.09876 0.03931 0.0708 -0.29171 -0.29171 0.124...,983367 8989128 16982545 24980703 32965997 4097...,89,27765788139068,1527873741034,...,1527873741067,-0.228638,0.070269,-0.050675,2.56126,3.89771,0.02285,1.60039,1.97426,0.15116
